# Neural Machine Translation Example

In [0]:
# Install TensorFlow and also our package via PyPI
!pip install tensorflow-gpu==2.0.0
!pip install headliner

     |████████████████████████████████| 380.8MB 68kB/s 
     |████████████████████████████████| 450kB 48.8MB/s 
     |████████████████████████████████| 3.8MB 38.0MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
     |████████████████████████████████| 51kB 4.2MB/s 


In [0]:
# Download the German-English sentence pairs
!wget http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
!head deu.txt

--2019-10-15 15:15:06--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7612057 (7.3M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   7.26M  2.18MB/s    in 3.3s    

2019-10-15 15:15:10 (2.18 MB/s) - ‘deu-eng.zip’ saved [7612057/7612057]

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              
Hi.	Hallo!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)
Hi.	Grüß Gott!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)
Run!	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)
Wow!	Potzdonner!	CC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #21

In [0]:
# Create the dataset but only take a subset for faster training
import io

def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

test = create_dataset('deu.txt', 30000)

eng, ger, _ = create_dataset('deu.txt', 30000)
data = list(zip(eng, ger))

In [0]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=100)

In [0]:
# Define the model and train it
from headliner.trainer import Trainer
from headliner.model.summarizer_attention import SummarizerAttention

summarizer = SummarizerAttention(lstm_size=1024, embedding_size=256)
trainer = Trainer(batch_size=64, 
                  steps_per_epoch=100, 
                  steps_to_log=20, 
                  max_output_len=10, 
                  model_save_path='/tmp/summarizer')
trainer.train(summarizer, train, num_epochs=10, val_data=test)

training a bare model, preprocessing data to init model...
fitting tokenizers...
vocab encoder: 4710, vocab decoder: 7528
epoch 0, batch 20, logs: {'loss': 4.143562316894531}
epoch 0, batch 40, logs: {'loss': 3.6472667694091796}
epoch 0, batch 60, logs: {'loss': 3.3786936442057294}
epoch 0, batch 80, logs: {'loss': 3.2139678955078126}
epoch 0, batch 100, logs: {'loss': 3.086736755371094}

(input) Thank you for this. 
(target) Danke dafür. 
(prediction) du ist ist nicht . <end>


(input) Do you live there? 
(target) Lebst du dort? 
(prediction) du ist ist ? <end>


(input) Nobody told me. 
(target) Das hat mir keiner gesagt. 
(prediction) ich habe ist nicht . <end>


(input) Is Tom bluffing? 
(target) Blufft Tom? 
(prediction) tom ist ist ? <end>


(input) You're worried. 
(target) Sie sind besorgt. 
(prediction) sie ist ist nicht . <end>

loss_val improved from None to 2.327824115753174, saving summarizer to /tmp/summarizer
epoch 1, batch 120, logs: {'loss': 2.4753684997558594, 'loss_v

In [0]:
# Do some prediction
summarizer.predict('How are you?')

'wie geht es dir ? <end>'